<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-MongoDB-Connection" data-toc-modified-id="Setup-MongoDB-Connection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup MongoDB Connection</a></span></li><li><span><a href="#Creating-a-dataframe-of-Manhattan-Buildings-(from-MongoDB)" data-toc-modified-id="Creating-a-dataframe-of-Manhattan-Buildings-(from-MongoDB)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a dataframe of Manhattan Buildings (from MongoDB)</a></span></li><li><span><a href="#Data-Analysis" data-toc-modified-id="Data-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Analysis</a></span><ul class="toc-item"><li><span><a href="#KMeans--Clustering" data-toc-modified-id="KMeans--Clustering-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>KMeans  Clustering</a></span></li><li><span><a href="#Try-Kmodes---to-cluster-on-categorical-variables" data-toc-modified-id="Try-Kmodes---to-cluster-on-categorical-variables-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Try <a href="https://github.com/nicodv/kmodes" target="_blank">Kmodes</a> - to cluster on categorical variables</a></span></li><li><span><a href="#Guassian-Mixture-Models" data-toc-modified-id="Guassian-Mixture-Models-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Guassian Mixture Models</a></span></li></ul></li></ul></div>

# K-Means Clustering of NYC Pluto Data
April 4th

## Setup MongoDB Connection

In [1]:
#load python dependencies
import pandas as pd
import pprint as pp
import json
import string
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import math

In [2]:
#connect to mongodb
from pymongo import MongoClient
client = MongoClient()
db = client.thesis #thesis is the name of the DB we want to access

In [3]:
# get the names of collections in MongoDB
db.collection_names(include_system_collections=False)

['buildings', 'buildings_with_lots', 'lots_with_buffer', 'green_roofs']

In [6]:
buildings = db.buildings

In [12]:
# spatial join test for 1 building
for doc in buildings.find(
    {
        "geometry": {
            "$geoWithin": {
                "$geometry" : {
                    "type" : "Polygon",
                     "coordinates" : [[[-74.00357149780405, 40.739750787571296],
                                       [-74.0032138954338,40.739665780439516],
                                       [-74.00320789710163,40.73966326774744],
                                       [-74.00320302205861,40.73965896338152],
                                       [-74.00319978576269,40.73965332245971],
                                       [-74.00319853040044,40.73964694141936],
                                       [-74.00319938870638,40.73964049495356],
                                       [-74.00320226992835,40.739634664673105],
                                       [-74.003246500287,40.739573008751414],
                                       [-74.00329109205792,40.73951084620651],
                                       [-74.00333404285205,40.73945097146967],
                                       [-74.00338030113019,40.73938648904522],
                                       [-74.00342944759335,40.73931797523731],
                                       [-74.00343423237918,40.739313246099265],
                                       [-74.00344031457,40.73931037105544],
                                       [-74.0034470059823,40.739309675409295],
                                       [-74.00345354950082,40.73931123787133],
                                       [-74.00384894205064,40.73947998532931],
                                       [-74.0038540011946,40.73948310469541],
                                       [-74.00385792023364,40.73948757308611],
                                       [-74.00386035306389,40.73949299588239],
                                       [-74.00386108483369,40.73949889417837],
                                       [-74.00386005091791,40.739504747075145],
                                       [-74.00385734262522,40.73951003768317],
                                       [-74.00367437802153,40.739765098595996],
                                       [-74.00367015922609,40.7397694166797],
                                       [-74.00366484415729,40.73977227921798],
                                       [-74.00365891707212,40.739773425404394],
                                       [-74.00365291798853,40.739772750809664],
                                       [-74.00357149780405,40.739750787571296]]]
                }
            }
        }
    }
):
    pp.pprint(doc)
    

{'_id': ObjectId('5ad118e98ebfd6b85610509d'),
 'geometry': {'coordinates': [[[-74.00339983290745, 40.73940048154231],
                               [-74.00340650854376, 40.739391313279995],
                               [-74.00341807222367, 40.739375430689684],
                               [-74.00342963234664, 40.73935955350131],
                               [-74.00343501167893, 40.739352164678394],
                               [-74.00358002094784, 40.7394132461763],
                               [-74.00359789020935, 40.73942077296012],
                               [-74.00359230963855, 40.73942843735214],
                               [-74.00355702913141, 40.739476894745856],
                               [-74.00355608206651, 40.73947819510768],
                               [-74.00355819422352, 40.739479084743664],
                               [-74.00361921593162, 40.739504787823364],
                               [-74.00364594443116, 40.73946807599871],
             

In [5]:
# Specify fields wanted
fields = {"properties.Borough" : 1,
            "properties.Address" : 1,
              "properties.BldgArea" : 1,
              "properties.BldgClass" : 1,
              "properties.Block" : 1,
              "properties.BoroCode" : 1,
              "properties.LandUse" : 1,
              "properties.LotArea" : 1,
              "properties.NumFloors" : 1,
              "properties.UnitsRes" : 1,
              "properties.UnitsTotal" : 1 ,
              "properties.OwnerName" : 1,
              "properties.YearBuilt": 1,
              "properties.CD" : 1,
              "properties.ResidFAR": 1,
              "properties.FacilFAR": 1, 
              "properties.BuiltFAR": 1,
              "properties.CommFAR": 1,
              "properties.Landmark": 1,
              "properties.UnitsTotal": 1,
              "properties.AssessTot": 1
          
         }
            

In [8]:
features.find_one({'properties.Borough':'MN'}, fields)

{'_id': ObjectId('5ab26407168e4a911bf9c754'),
 'properties': {'Address': '263 9 AVENUE',
  'AssessTot': 10493435,
  'BldgArea': 89203,
  'BldgClass': 'R4',
  'Block': 723,
  'BoroCode': 1,
  'Borough': 'MN',
  'BuiltFAR': 9.03,
  'CD': 104,
  'FacilFAR': 6.5,
  'LandUse': '03',
  'Landmark': None,
  'LotArea': 9875,
  'NumFloors': 11,
  'OwnerName': None,
  'ResidFAR': 6.02,
  'UnitsRes': 50,
  'UnitsTotal': 50,
  'YearBuilt': 1914}}

## Creating a dataframe of Manhattan Buildings (from MongoDB)

In [6]:
def mongo_to_dataframe(mongo_data):

        sanitized = json.loads(json_util.dumps(mongo_data))
        
        # unnests the json data
        normalized = json_normalize(sanitized)
        
        #creates a data frame
        df = pd.DataFrame(normalized)
        
        #cleans up column names
        df.columns = df.columns.str.replace('properties.', '')

        return df

In [9]:
df = mongo_to_dataframe(features.find({'properties.Borough':'MN'}, fields))

In [1]:
print(df.shape)
df.head()

NameError: name 'df' is not defined